In [0]:
dbutils.widgets.text(name="env", defaultValue="", label="Enter environment")
env = dbutils.widgets.get("env")
env

'dev'

In [0]:
from pyspark.sql import DataFrame
from functools import reduce
from dataclasses import dataclass
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import current_timestamp, col

In [0]:
%run "./paths"

('abfss://landing@dlsunitycat.dfs.core.windows.net/',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/bronze',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/silver',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/gold')

In [0]:
table = 'state_gdp'
path = raw_paths[table]


In [0]:
df = spark.read.csv(path, header=True, inferSchema=True)


In [0]:
non_quarter_columns = [
    "GeoFIPS", "GeoName", "Region", "TableName", "LineCode",
    "IndustryClassification", "Description", "Unit"
]

quarter_columns = [col for col in df.columns if ":" in col]

df_unpivoted = (df.selectExpr(
    *non_quarter_columns,
    "stack({}, {}) as (business_year, quarter, value)".format(
        len(quarter_columns),
        ", ".join([f"'{col.split(':')[0]}', '{col.split(':')[1]}', `{col}`" for col in quarter_columns])
    )
).withColumn("business_year", col("business_year").cast("int"))
.filter(col('Unit') == 'Millions of current dollars')
)

df_unpivoted = df_unpivoted.withColumn("date_ingested", current_timestamp())

df_unpivoted.write.format("delta").mode("overwrite").save(bronze_paths[table])
